# **Fase di Modellazione**

## **1. Introduzione**
In questa fase, lavoriamo con il dataset preprocessato per creare un modello di machine learning in grado di predire la presenza di malattie cardiovascolari. I passaggi principali includono:
1. **Caricamento del dataset preprocessato.**
2. **Selezione degli algoritmi di machine learning.**
3. **Addestramento e configurazione dei modelli.**
4. **Valutazione delle performance tramite benchmarking.**


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Caricamento del dataset preprocessato
data = pd.read_csv('prepared_cardio_train.csv')

# Separazione delle feature e del target
X = data.drop('cardio', axis=1)
y = data['cardio']

# Divisione in training e test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Inizializzazione dei modelli
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Random Forest': RandomForestClassifier(random_state=42),
    'SVM': SVC(random_state=42),
    'KNN': KNeighborsClassifier()
}

# Addestramento e valutazione
results = []

for model_name, model in models.items():
    # Addestramento
    model.fit(X_train, y_train)
    # Predizioni
    y_pred = model.predict(X_test)
    # Valutazione
    results.append({
        'Model': model_name,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1 Score': f1_score(y_test, y_pred)
    })

# Creazione di un DataFrame con i risultati
results_df = pd.DataFrame(results).sort_values(by='F1 Score', ascending=False)
print(results_df)


                 Model  Accuracy  Precision    Recall  F1 Score
2                  SVM  0.730521   0.761557  0.663754  0.709300
0  Logistic Regression  0.725972   0.751753  0.667012  0.706852
3                  KNN  0.696185   0.696536  0.685084  0.690762
1        Random Forest  0.683786   0.677064  0.691305  0.684110


# **Selezione e Ottimizzazione del Modello**

## **1. Modello Selezionato**
Dopo aver confrontato diversi algoritmi, abbiamo selezionato il modello **SVM** come il migliore per il problema di predizione delle malattie cardiovascolari. Il modello ha ottenuto il punteggio più alto in termini di F1 Score (0.7093), mostrando un buon equilibrio tra precision e recall.

## **2. Obiettivo**
Ora procediamo con l'ottimizzazione dei parametri del modello SVM utilizzando la tecnica di Grid Search per trovare la configurazione ottimale.


In [4]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from tqdm import tqdm
from joblib import Parallel, delayed
import sys

class TQDMGridSearchCV(GridSearchCV):
    def fit(self, X, y=None, **fit_params):
        total = len(self.param_grid['C']) * len(self.param_grid['kernel']) * len(self.param_grid['gamma']) * len(self.param_grid.get('degree', [1])) * self.cv
        self.pbar = tqdm(total=total, desc="GridSearch Progress", file=sys.stdout)
        
        def _parallel_fit(*args, **kwargs):
            result = super()._parallel_fit(*args, **kwargs)
            self.pbar.update(1)
            return result
        
        # Override the parallel_fit method
        self._parallel_fit = _parallel_fit
        return super().fit(X, y, **fit_params)

# Definizione del modello SVM
svm_model = SVC(random_state=42)

# Definizione della griglia di parametri
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto'],
    'degree': [2, 3, 4]  # Usato solo per kernel 'poly'
}

# Inizializzazione della Grid Search con la classe personalizzata
grid_search = TQDMGridSearchCV(
    estimator=svm_model,
    param_grid=param_grid,
    scoring='f1',
    cv=5,
    verbose=0,
    n_jobs=-1
)

# Addestramento del modello con Grid Search
grid_search.fit(X_train, y_train)

# Chiudere la barra di avanzamento
grid_search.pbar.close()

# Miglior modello e parametri
best_model = grid_search.best_estimator_
print("Miglior modello:", grid_search.best_params_)

# Valutazione del modello ottimizzato
y_pred_optimized = best_model.predict(X_test)
print("\nReport di classificazione per il modello ottimizzato:")
print(classification_report(y_test, y_pred_optimized))


GridSearch Progress:   0%|          | 0/270 [00:00<?, ?it/s]

GridSearch Progress:   0%|          | 0/270 [2:14:42<?, ?it/s]
Miglior modello: {'C': 1, 'degree': 2, 'gamma': 'auto', 'kernel': 'rbf'}

Report di classificazione per il modello ottimizzato:
              precision    recall  f1-score   support

           0       0.71      0.80      0.75      6879
           1       0.76      0.66      0.71      6751

    accuracy                           0.73     13630
   macro avg       0.73      0.73      0.73     13630
weighted avg       0.73      0.73      0.73     13630

